# PROBLEM 2
Magic Keys given in “problem 2.csv” purchased only one box of milk and/or meat in the first 15 days of March-2019. You need to predict which boxes were purchased by these customers in this period.

In [444]:
import pandas as pd

## Exploration

In [445]:
purchase_df = pd.read_csv("purchase.csv")
boxes_df = pd.read_csv("boxes.csv")
problem2_df = pd.read_csv("problem 2.csv")

In [446]:
# Utility Function

def analyze(df):
    print(f"Number of rows: {len(df)}")    
    print(f"Number of columns: {len(df.columns)}")
    
    info_df = pd.DataFrame({
        "NUNIQUE": df.nunique(),
        "DTYPE": df.dtypes
    })
    
    print()
    print(info_df)

In [447]:
analyze(purchase_df)

Number of rows: 2455864
Number of columns: 4

               NUNIQUE    DTYPE
PURCHASE_DATE      151   object
MAGIC_KEY      1274108   object
BOX_ID             291  float64
BOX_COUNT           14  float64


In [448]:
analyze(boxes_df)

Number of rows: 290
Number of columns: 6

                 NUNIQUE    DTYPE
BOX_ID               290    int64
QUALITY                2   object
DELIVERY_OPTION        3   object
MILK                  28  float64
MEAT                  39  float64
UNIT_PRICE            18  float64


In [449]:
analyze(problem2_df)

Number of rows: 43518
Number of columns: 1

           NUNIQUE   DTYPE
MAGIC_KEY    43518  object


## Preprocessing

In [450]:
merged_df = purchase_df.merge(boxes_df, on="BOX_ID", how="left")

In [451]:
analyze(merged_df)

Number of rows: 2455864
Number of columns: 9

                 NUNIQUE    DTYPE
PURCHASE_DATE        151   object
MAGIC_KEY        1274108   object
BOX_ID               291  float64
BOX_COUNT             14  float64
QUALITY                2   object
DELIVERY_OPTION        3   object
MILK                  28  float64
MEAT                  39  float64
UNIT_PRICE            18  float64


In [452]:
merged_df["PURCHASE_DATE"] = pd.to_datetime(merged_df['PURCHASE_DATE'],format='%d/%m/%Y')
merged_df['DAY'] = merged_df['PURCHASE_DATE'].dt.day
merged_df['MONTH'] = merged_df['PURCHASE_DATE'].dt.month
merged_df['YEAR'] = merged_df['PURCHASE_DATE'].dt.year

In [453]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

merged_df["QUALITY"] = le.fit_transform(merged_df["QUALITY"])
merged_df["DELIVERY_OPTION"] = le.fit_transform(merged_df["DELIVERY_OPTION"])
merged_df["ALT_MAGIC_KEY"] = le.fit_transform(merged_df["MAGIC_KEY"])

In [454]:
merged_df.drop(columns=["PURCHASE_DATE"], inplace=True)
merged_df = merged_df.reindex(columns=["MAGIC_KEY", "ALT_MAGIC_KEY",
                                       "YEAR", "MONTH", "DAY",
                                       "MILK", "MEAT",
                                       "BOX_COUNT", "UNIT_PRICE",
                                       "QUALITY", "DELIVERY_OPTION",
                                       "BOX_ID"])

In [455]:
analyze(merged_df)

Number of rows: 2455864
Number of columns: 12

                 NUNIQUE    DTYPE
MAGIC_KEY        1274108   object
ALT_MAGIC_KEY    1274108    int32
YEAR                   2    int32
MONTH                  5    int32
DAY                   31    int32
MILK                  28  float64
MEAT                  39  float64
BOX_COUNT             14  float64
UNIT_PRICE            18  float64
QUALITY                3    int32
DELIVERY_OPTION        4    int32
BOX_ID               291  float64


In [456]:
merged_df.isna().sum()

MAGIC_KEY           0
ALT_MAGIC_KEY       0
YEAR                0
MONTH               0
DAY                 0
MILK               64
MEAT               64
BOX_COUNT          47
UNIT_PRICE         64
QUALITY             0
DELIVERY_OPTION     0
BOX_ID             47
dtype: int64

In [457]:
merged_df = merged_df[merged_df["BOX_ID"].isna() == False]

In [458]:
merged_df.isna().sum()

MAGIC_KEY           0
ALT_MAGIC_KEY       0
YEAR                0
MONTH               0
DAY                 0
MILK               17
MEAT               17
BOX_COUNT           0
UNIT_PRICE         17
QUALITY             0
DELIVERY_OPTION     0
BOX_ID              0
dtype: int64

In [459]:
merged_df[merged_df["MILK"].isna()]

,MAGIC_KEY,ALT_MAGIC_KEY,YEAR,MONTH,DAY,MILK,MEAT,BOX_COUNT,UNIT_PRICE,QUALITY,DELIVERY_OPTION,BOX_ID
191,2CCB122460D,1110382,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
192,2CEDA71E3E9,1223075,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
193,2CBDC970258,1067979,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
194,2932761FEB6,234644,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
195,28F09714410,96773,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
196,2BDAE37A00E,429999,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
197,2C3C7E168F7,735354,2019,2,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
525713,2C3C7E168F7,735354,2019,2,27,NaN,NaN,-1.0,NaN,2,3,11111111.0
597615,2CEDA71E3E9,1223075,2019,1,1,NaN,NaN,-1.0,NaN,2,3,11111111.0
665861,2C3C7E168F7,735354,2019,1,5,NaN,NaN,-1.0,NaN,2,3,11111111.0


In [460]:
merged_df = merged_df[merged_df["BOX_COUNT"] > 0]

In [461]:
merged_df.isna().sum()

MAGIC_KEY          0
ALT_MAGIC_KEY      0
YEAR               0
MONTH              0
DAY                0
MILK               0
MEAT               0
BOX_COUNT          0
UNIT_PRICE         0
QUALITY            0
DELIVERY_OPTION    0
BOX_ID             0
dtype: int64

In [462]:
analyze(merged_df)

Number of rows: 2455800
Number of columns: 12

                 NUNIQUE    DTYPE
MAGIC_KEY        1274087   object
ALT_MAGIC_KEY    1274087    int32
YEAR                   2    int32
MONTH                  5    int32
DAY                   31    int32
MILK                  28  float64
MEAT                  39  float64
BOX_COUNT             13  float64
UNIT_PRICE            18  float64
QUALITY                2    int32
DELIVERY_OPTION        3    int32
BOX_ID               290  float64


In [463]:
merged_df.head(10)

,MAGIC_KEY,ALT_MAGIC_KEY,YEAR,MONTH,DAY,MILK,MEAT,BOX_COUNT,UNIT_PRICE,QUALITY,DELIVERY_OPTION,BOX_ID
0,2CED678A247,1222234,2019,2,1,8.0,1.5,1.0,12.98,0,1,12.0
1,2BF58D91BA1,526391,2019,2,1,8.0,1.5,1.0,12.98,0,1,12.0
2,2C15B86534E,624102,2019,2,1,0.0,3.3,1.0,13.96,0,0,99.0
3,2C32D9A859A,708928,2019,2,1,0.0,2.7,1.0,11.96,0,1,6.0
4,2C7A55404D1,870797,2019,2,1,0.0,2.5,1.0,11.96,0,1,4.0
5,29D969045C2,310429,2019,2,1,10.7,0.0,1.0,12.78,1,0,238.0
6,28E5EA49074,65441,2019,2,1,8.1,0.0,1.0,9.96,1,0,227.0
7,2CEFA3A8659,1230129,2019,2,1,0.0,2.7,1.0,11.96,0,1,6.0
8,2A00DE30F46,326681,2019,2,1,10.7,0.0,1.0,12.78,1,1,204.0
9,291C04B5CBF,214844,2019,2,1,8.5,0.0,1.0,10.14,1,0,231.0


In [464]:
# filtered_df = merged_df[(merged_df["DAY"] <= 15) & (merged_df["BOX_COUNT"] == 1)]
filtered_df = merged_df[merged_df["BOX_COUNT"] == 1]
filtered_df = filtered_df.drop_duplicates(subset=["MAGIC_KEY"])

In [465]:
problem2_df["MAGIC_KEY"].isin(filtered_df["MAGIC_KEY"]).all()

True

In [466]:
analyze(problem2_df)

Number of rows: 43518
Number of columns: 1

           NUNIQUE   DTYPE
MAGIC_KEY    43518  object


In [467]:
analyze(filtered_df)

Number of rows: 1273464
Number of columns: 12

                 NUNIQUE    DTYPE
MAGIC_KEY        1273464   object
ALT_MAGIC_KEY    1273464    int32
YEAR                   2    int32
MONTH                  5    int32
DAY                   31    int32
MILK                  25  float64
MEAT                  33  float64
BOX_COUNT              1  float64
UNIT_PRICE            17  float64
QUALITY                2    int32
DELIVERY_OPTION        3    int32
BOX_ID               232  float64


## Model Training

In [468]:
problem_data = pd.merge(problem2_df, filtered_df, on="MAGIC_KEY", how='left')
problem_data.fillna(0, inplace=True)

In [469]:
problem_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43518 entries, 0 to 43517
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MAGIC_KEY        43518 non-null  object 
 1   ALT_MAGIC_KEY    43518 non-null  int32  
 2   YEAR             43518 non-null  int32  
 3   MONTH            43518 non-null  int32  
 4   DAY              43518 non-null  int32  
 5   MILK             43518 non-null  float64
 6   MEAT             43518 non-null  float64
 7   BOX_COUNT        43518 non-null  float64
 8   UNIT_PRICE       43518 non-null  float64
 9   QUALITY          43518 non-null  int32  
 10  DELIVERY_OPTION  43518 non-null  int32  
 11  BOX_ID           43518 non-null  float64
dtypes: float64(5), int32(6), object(1)
memory usage: 3.0+ MB


In [472]:
problem_data.nunique()

MAGIC_KEY          43518
ALT_MAGIC_KEY      43518
YEAR                   2
MONTH                  3
DAY                   31
MILK                  22
MEAT                  18
BOX_COUNT              1
UNIT_PRICE            16
QUALITY                2
DELIVERY_OPTION        3
BOX_ID               127
dtype: int64

In [473]:
from sklearn.model_selection import train_test_split

X = problem_data.drop(["MAGIC_KEY", "BOX_ID"], axis=1)
y = problem_data["BOX_ID"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [474]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [475]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 99.83%


In [476]:
prediction = model.predict(X)

In [477]:
submission = pd.DataFrame({
    'MAGIC_KEY': problem2_df['MAGIC_KEY'],
    'BOX_ID': [pred for pred in prediction ]
})

In [478]:
submission.to_csv("submission.csv", index=False)